In [ ]:
import $ivy.`org.apache.spark:spark-sql_2.13:3.5.1`
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._

val spark = SparkSession.builder()
  .appName("Ques14")
  .master("local[*]")  // Use all cores
  .getOrCreate()

// Create a DataFrame from a sequence of data
import spark.implicits._

val schema = StructType(Seq(
    StructField("locality", IntegerType, true),
    StructField("type_of_property", StringType, true),
    StructField("subtype_of_property", StringType, true),
    StructField("price", IntegerType, true),
    StructField("no_of_rooms", IntegerType, true),
    StructField("open_fire", IntegerType, true),
    StructField("terrace", IntegerType, true),
    StructField("garden", IntegerType, true),
    StructField("swimming_pool", IntegerType, true),
    StructField("state_of_building", StringType, true),
    StructField("construction_year", IntegerType, true)
))

val haDF = spark.read.option("header","false").schema(schema).csv("house_apartment_cleaned_data.csv")
val filter_df = haDF.filter($"type_of_property" === "house")
val group_df = filter_df.groupBy("subtype_of_property", "construction_year").agg(sum("price").alias("total_price"))
val pivot = group_df.groupBy("subtype_of_property").pivot("construction_year", Seq(1990, 2020)).agg(first("total_price"))
val pivot_df = pivot.na.fill(Map("1990" -> 0, "2020" -> 0))
val final_df = pivot_df.withColumn("price_difference", abs(col("2020") - col("1990")))
final_df.show()
final_df.rdd.getNumPartitions
final_df.write.option("header","true").csv("./Que__10")